In [5]:
from newspaper import build
from newspaper import Article
from datetime import datetime, timedelta
import json
import time
import glob
import os
import traceback
import requests
import nltk
from bs4 import BeautifulSoup, SoupStrainer
#nltk.download('punkt')

#outputdir = '/opt/newspaperdata/news/'
outputdir = '/tmp/'

inputfile= 'list-websites.txt'
listArticles = []
publisher=''



class articleObj:
    def __init__(self, title, date_published, news_outlet,
                 feature_img, article_link, keywords,movies, Summary, text):
        self.title = title
        self.date_published = date_published
        self.news_outlet = news_outlet
        self.feature_img = feature_img
        self.article_link = article_link
        self.keywords = keywords
        self.movies = movies
        self.Summary = Summary
        self.text = text
     

def website_parser (article, publisher, dateToday):
    movies =[]
    date_published = dateToday
    title = article.title,
    website = publisher
    feature_img =  article.top_image
    article_link = article.url
    keywords = article.keywords
    
    #extract videos
    r = requests.get(article.url)
    soup = BeautifulSoup(r.content,'html.parser')
    links = soup.find_all('iframe')
    for link in links:
        try:
            video_link = link['src']
            #video_link = link['video']
        except Exception:
            continue
        if video_link:
            if  not(('googletagmanager' in video_link) or ('ServiceLogin' in video_link)) :
                print('video_link '+video_link)
                movies.append(video_link)
    summary = article.summary
    text = article.text
    anArticle= articleObj (title, date_published, website, 
                                    feature_img, article_link, keywords,
                                    movies, summary, text)
    listArticles.append(anArticle)

def write_to_json (file_name):
    with open(file_name, 'w') as f:
        if listArticles:
            json.dump([ob.__dict__ for ob in listArticles], f)

def scrape_url ():
    with open(inputfile) as f:
         lines = [line.rstrip() for line in f]
    for line in lines:
        publisher=''
        global listArticles
        listArticles = []
        i = 0
        append_url = ''
        base_url = ''
        
        articles = []
        page = requests.get(line)    
        data = page.text
        soup = BeautifulSoup(data)
        exclude = ['/a-z', '/LoginPage', '/privacy', '/support', 
                   '/contact-us','/alexa','/referrals','/browse/','/help',
                   '/sitemap','/cookies','/language-policy','/personal-data-protection',
                   '/copyright','/our-policies','/all-topics','/news-events','/publications-data',
                   '/data-tools','about-us','mailto','/terms-conditions','/privacy-notice',
                   '/working','/visas-immigration','/abroad','/tax','/housing-local-services',
                   '/environment-countryside','/employing-people','/education','/driving','/disabilities',
                   '/justice','/citizenship','/childcare-parenting','/business','/births-deaths-marriages',
                   '/benefits','/contact','/terms-conditions','/who-brochure','/footer','/nhs-sites/','/login','sitemap/',
                   '/accessibility-statement/','/our-policies/','/jobs','/libraries','/redirect-pages']

        for link in soup.find_all('a'):
            alink = link.get('href')
            if not any(ex in alink for ex in exclude):
                articles.append(alink)
        articles.append(line)
                
        if line == 'https://www.gov.uk/coronavirus' :
            base_url = 'https://www.gov.uk'
        elif line == 'https://www.nhs.uk/conditions/coronavirus-covid-19' :
            base_url = 'https://www.nhs.uk'
        elif line == 'https://www.gov.uk/government/organisations/scientific-advisory-group-for-emergencies' :
            base_url = 'https://www.gov.uk'
        elif line == 'https://www.cdc.gov/coronavirus/2019-ncov/index.html' :
            base_url = 'https://www.cdc.gov'
        elif line == 'https://www.who.int/emergencies/diseases/novel-coronavirus-2019':
            base_url = 'https://www.who.int'
        elif line == 'https://www.nhs.uk/conditions/coronavirus-covid-19' :
            base_url = 'https://www.nhs.uk'
        else :
            base_url = line
            
        for article in articles:
            if article : 
                if  not(('http://'  in article) or ('https://' in article)) :
                    article = base_url + article 
            print('URL '+article)
            article = Article(article)
            try:
                article.build()
            except Exception:
                print(traceback.format_exc())
                #time.sleep(60)
                continue
            data = website_parser(article, line, dateToday)
        write_to_json('/tmp/'+dateNow+'/'+line.replace('https://www','').replace('.','').replace('/','')
                      .replace(':','').replace('https','')
                      +datetime.today().strftime('%Y-%m-%d').replace(' ','T')+'.json')
        
        
dateNow = datetime.now().strftime("%H%M%S%f")
dateToday = datetime.today().strftime('%Y-%m-%d')
if  not os.path.exists('/tmp/'+dateNow):
    os.mkdir('/tmp/'+dateNow)
scrape_url()
glob_data = []
for file in glob.glob('/tmp/'+dateNow+'/'+'*.json'):
    if os.path.getsize(file) > 0 :
        with open(file) as json_file:
            data = json.load(json_file)
            i = 0
            while i < len(data):
                glob_data.append(data[i])
                i += 1              
with open(outputdir+dateToday+'.json', 'w') as f:
    json.dump(glob_data, f, indent=4)
    
        


URL https://www.gov.uk#content
URL https://www.gov.uk
URL https://www.gov.uk/coronavirus
URL https://www.gov.uk/transition
URL https://www.gov.uk/
URL https://www.gov.uk/guidance/new-national-restrictions-from-5-november
URL https://www.gov.uk/get-coronavirus-test
URL https://www.gov.uk/find-coronavirus-support
URL https://www.gov.uk/government/publications/making-a-christmas-bubble-with-friends-and-family/making-a-christmas-bubble-with-friends-and-family
URL https://www.gov.uk/find-coronavirus-local-restrictions
URL https://www.gov.uk/government/publications/covid-19-winter-plan
URL https://www.gov.uk/guidance/new-national-restrictions-from-5-november
URL https://www.gov.uk/guidance/new-national-restrictions-from-5-november?priority-taxon=774cee22-d896-44c1-a611-e3109cce8eae
URL https://www.gov.uk/government/publications/coronavirus-covid-19-meeting-with-others-safely-social-distancing/coronavirus-covid-19-meeting-with-others-safely-social-distancing?priority-taxon=774cee22-d896-44c1-

URL https://liverpool.gov.uk/bins-and-recycling/bulky-item-collection/
URL https://liverpool.gov.uk/leisure-parks-and-wellbeing/
URL https://liverpool.gov.uk/lifestyles/
URL https://liverpool.gov.uk/leisure-parks-and-wellbeing/parks-and-greenspaces/
URL https://liverpool.gov.uk/leisure-parks-and-wellbeing/events/
URL https://liverpool.gov.uk/council-tax/
URL https://liverpool.gov.uk/council-tax/register-for-council-tax-or-tell-us-about-a-change/
URL https://liverpool.gov.uk/council-tax/how-to-pay/
URL https://liverpool.gov.uk/council-tax/how-much/
URL https://liverpool.gov.uk/parking-roads-and-travel/
URL https://liverpool.gov.uk/parking-roads-and-travel/parking-permits/
URL https://liverpool.gov.uk/parking-roads-and-travel/find-parking/
URL https://liverpool.gov.uk/parking-roads-and-travel/parking-fines-and-challenges/
URL https://liverpool.gov.uk/council/
URL https://cityplanliverpool.co.uk/
video_link https://vimeo.com/event/343041/embed
URL https://liverpool.gov.uk/council/councill

URL https://www.gov.uk/search/transparency-and-freedom-of-information-releases?organisations%5B%5D=scientific-advisory-group-for-emergencies&parent=scientific-advisory-group-for-emergencies
URL https://www.gov.uk/make-a-freedom-of-information-request/the-freedom-of-information-act
URL https://www.gov.uk/government/publications?departments%5B%5D=scientific-advisory-group-for-emergencies&publication_type=foi-releases
URL https://www.civilservicejobs.service.gov.uk/csr
URL https://www.gov.uk/coronavirus
URL https://www.gov.uk/transition
URL https://www.gov.uk/government/how-government-works
URL https://www.gov.uk/government/organisations
URL https://www.gov.uk/world
URL https://www.gov.uk/search/services
URL https://www.gov.uk/search/guidance-and-regulation
URL https://www.gov.uk/search/news-and-communications
URL https://www.gov.uk/search/research-and-statistics
URL https://www.gov.uk/search/policy-papers-and-consultations
URL https://www.gov.uk/search/transparency-and-freedom-of-informa

URL https://www.cdc.gov#az-large
URL https://www.cdc.gov
URL https://www.cdc.gov#
URL https://www.cdc.gov#
URL https://www.cdc.gov/az/
URL https://www.cdc.gov#cdcAdvancedSearch
URL https://www.cdc.gov#cdcAdvancedSearch
URL https://www.youtube.com/playlist?list=PLvrp9iOILTQatwnqm61jqFrsfUB4RKh6J
URL https://www.cdc.gov#
URL https://espanol.cdc.gov/enes/coronavirus/2019-ncov/index.html
URL https://chinese.cdc.gov/coronavirus/2019-ncov/index.html
URL https://vietnamese.cdc.gov/coronavirus/2019-ncov/index.html
URL https://korean.cdc.gov/coronavirus/2019-ncov/index.html
URL https://wwwn.cdc.gov/pubs/other-languages/
URL https://www.cdc.gov/coronavirus/2019-ncov/index.html
URL https://www.cdc.gov#
URL https://www.cdc.gov/coronavirus/2019-ncov/your-health/need-to-know.html
URL https://www.cdc.gov/coronavirus/2019-ncov/symptoms-testing/symptoms.html
URL https://www.cdc.gov/coronavirus/2019-ncov/testing/index.html
URL https://www.cdc.gov/coronavirus/2019-ncov/vaccines/index.html
URL https://www

URL https://www.cdc.gov/coronavirus/2019-ncov/daily-life-coping/holidays.html
URL https://www.cdc.gov/coronavirus/2019-ncov/symptoms-testing/symptoms.html
URL https://covid19healthbot.cdc.gov/
URL https://www.cdc.gov/coronavirus/2019-ncov/symptoms-testing/testing.html
video_link //www.youtube-nocookie.com/embed/gff4GkGvW-Q?autohide=0&enablejsapi=1&playerapiid=14865&modestbranding=1&rel=0&origin=https://www.cdc.gov?autohide=0&enablejsapi=1&playerapiid=14865&modestbranding=1&rel=0&origin=https://www.cdc.gov&wmode=opaque?enablejsapi=1
URL https://www.cdc.gov/coronavirus/2019-ncov/vaccines/index.html
URL https://www.cdc.gov/coronavirus/2019-ncov/if-you-are-sick/steps-when-sick.html
URL https://www.cdc.gov/coronavirus/2019-ncov/if-you-are-sick/isolation.html
video_link //www.youtube-nocookie.com/embed/l3s75_X8Xjs?autohide=0&enablejsapi=1&playerapiid=88323&modestbranding=1&rel=0&origin=https://www.cdc.gov?autohide=0&enablejsapi=1&playerapiid=88323&modestbranding=1&rel=0&origin=https://www.cd

URL https://espanol.cdc.gov/enes/coronavirus/2019-ncov/index.html
URL https://chinese.cdc.gov/coronavirus/2019-ncov/index.html
URL https://vietnamese.cdc.gov/coronavirus/2019-ncov/index.html
URL https://korean.cdc.gov/coronavirus/2019-ncov/index.html
URL https://wwwn.cdc.gov/pubs/other-languages?Sort=Lang%3A%3Aasc
URL https://www.coronavirus.gov/
URL https://www.usa.gov/coronavirus
URL https://www.cdc.gov/cdc-info/index.html
URL https://www.cdc.gov/cdc-info/index.html
URL https://www.cdc.gov/about/default.htm
video_link //www.youtube-nocookie.com/embed/RmolrQoUpik?autohide=0&enablejsapi=1&playerapiid=3587&modestbranding=1&rel=0&origin=https://www.cdc.gov&showinfo=0//www.youtube-nocookie.com/embed/RmolrQoUpik?autohide=0&enablejsapi=1&playerapiid=3587&modestbranding=1&rel=0&origin=https://www.cdc.gov&showinfo=0&wmode=opaque?enablejsapi=1
video_link //www.youtube-nocookie.com/embed/IsITU2RLEo4?autohide=0&enablejsapi=1&playerapiid=3587&modestbranding=1&rel=0&origin=https://www.cdc.gov?auto

URL https://www.ecdc.europa.eu/en/immunisation-and-vaccines
URL https://flunewseurope.org/
URL https://www.ecdc.europa.eu/en/hiv-infection-and-aids
URL https://www.ecdc.europa.eu/en/viral-hepatitis
URL https://www.ecdc.europa.eu/en/tuberculosis
URL https://www.ecdc.europa.eu/en/about-ecdc
URL https://www.ecdc.europa.eu/en/vacancies?f%5B0%5D=deadline_date%3A1
URL https://www.ecdc.europa.eu/en/pressroom
URL http://extranet.ecdc.europa.eu
Traceback (most recent call last):
  File "<ipython-input-5-47d9403dd80b>", line 126, in scrape_url
    article.build()
  File "/usr/Newspaper-Scrapers/newspaper/article.py", line 162, in build
    self.parse()
  File "/usr/Newspaper-Scrapers/newspaper/article.py", line 213, in parse
    self.throw_if_not_downloaded_verbose()
  File "/usr/Newspaper-Scrapers/newspaper/article.py", line 560, in throw_if_not_downloaded_verbose
    (self.download_exception_msg, self.url))
newspaper.article.ArticleException: Article `download()` failed with 403 Client Error: 

video_link https://www.youtube.com/embed/yEIPefMsf70
URL https://www.who.int/
video_link https://www.youtube.com/embed/yEIPefMsf70
URL https://www.who.int/health-topics/
URL https://www.who.int/health-topics/#A
URL https://www.who.int/health-topics/#B
URL https://www.who.int/health-topics/#C
URL https://www.who.int/health-topics/#D
URL https://www.who.int/health-topics/#E
URL https://www.who.int/health-topics/#F
URL https://www.who.int/health-topics/#G
URL https://www.who.int/health-topics/#H
URL https://www.who.int/health-topics/#I
URL https://www.who.int/health-topics/#J
URL https://www.who.int/health-topics/#K
URL https://www.who.int/health-topics/#L
URL https://www.who.int/health-topics/#M
URL https://www.who.int/health-topics/#N
URL https://www.who.int/health-topics/#O
URL https://www.who.int/health-topics/#P
URL https://www.who.int/health-topics/#Q
URL https://www.who.int/health-topics/#R
URL https://www.who.int/health-topics/#S
URL https://www.who.int/health-topics/#T
URL https:

URL https://app.powerbi.com/view?r=eyJrIjoiNzNmNTRkMWEtNmZjMS00NzdjLWEyMDYtYWExYzA4NzVhZGQwIiwidCI6ImY2MTBjMGI3LWJkMjQtNGIzOS04MTBiLTNkYzI4MGFmYjU5MCIsImMiOjh9
URL https://www.who.int/emergencies/diseases/novel-coronavirus-2019/who-response-in-countries
URL https://www.who.int/emergencies/partners
URL http://www.un.org/coronavirus
video_link https://www.youtube.com/embed/a7Oe4bOMBws
video_link https://www.youtube.com/embed/yEIPefMsf70
video_link https://www.youtube.com/embed/?list=PL9S6xGsoqIBXRQzSDOfFb13iPrbL8fgy1&?rel=0
video_link https://www.youtube.com/embed/Ywj9DtrI-EM?rel=0
URL https://www.who.int/emergencies/diseases/novel-coronavirus-2019/who-director-general-s-special-envoys-on-covid-19-preparedness-and-response
URL https://www.who.int/emergencies/diseases/novel-coronavirus-2019/situation-reports
URL https://www.who.int/blueprint/priority-diseases/key-action/novel-coronavirus/en/
URL https://extranet.who.int/publicemergency
URL https://www.who.int/emergencies/diseases/novel-co

In [17]:
# -*- coding = utf-8 -*-

import urllib, sys
from BeautifulSoup import BeautifulSoup # for html

def fetch_url(link):
	''' '''
	f = urllib.urlopen(link)
	contents = f.read()	
	f.close()
	
	'''
	f.open(filename, 'w')
	f.write(contents)
	f.close()
	'''
	
	return BeautifulSoup(''.join(contents))
	
def listLinks(soup):
	''' '''
	links = []
	for link in soup.findAll('a', rel = 'lecture-link'):
		links.append(link.get('href'))	
	return links
	
def findMP4link(link):
	''' '''	
	opener = urllib.build_opener()
	f = opener.open(link)
	contents = f.read()
	f.close()
	opener.close()
		
	soup = BeautifulSoup(''.join(contents))
	links = soup.findAll('source', type = 'video/mp4')
	assert len(links) == 1
	file = links[0]	
	return file.get('src')
	
	'''
	files = []
	for file in soup.findAll('source', type = 'video/mp4'):
		files.append(file.get('src'))
	
	return files
	'''
	
if __name__ == "__main__":
	''' '''
	# for gpm
	# link = "https://class.coursera.org/pgm/lecture/preview"
	# soup = BeautifulSoup(''.join(contents))
	
	# for ml
	# link = "https://class.coursera.org/ml/lecture/preview"
	
	# for nlp
	link = "https://class.coursera.org/nlp/lecture/preview"
	
	soup = fetch_url(link)
	# f = open('contents.html', 'r')
	# contents = f.read()
	# f.close()
		
	# print soup.prettify()	
	links = listLinks(soup)
	# print links	
	f = open('nlp', 'w')
	for link in links:		
		mp4_link = findMP4link(link)		
		f.write(mp4_link)
		f.write("\n")
	f.close()

ModuleNotFoundError: No module named 'BeautifulSoup'

In [42]:
from newspaper import build
from newspaper import Article
from datetime import datetime, timedelta
article = Article('https://www.who.int/emergencies/diseases/novel-coronavirus-2019/advice-for-public/videos')
try:
    article.build()
except Exception:
    print(traceback.format_exc())
print(article.url)

https://www.who.int/emergencies/diseases/novel-coronavirus-2019/advice-for-public/videos


In [19]:
import urllib, urllib, sys
from bs4 import BeautifulSoup, SoupStrainer

def fetch_url(link):
	''' '''
	f = urllib.urlopen(link)
	contents = f.read()	
	f.close()
	
	'''
	f.open(filename, 'w')
	f.write(contents)
	f.close()
	'''
	
	return BeautifulSoup(''.join(contents))
	
def listLinks(soup):
	''' '''
	links = []
	for link in soup.findAll('a', rel = 'lecture-link'):
		links.append(link.get('href'))	
	return links
	
def findMP4link(link):
	''' '''	
	opener = urllib.build_opener()
	f = opener.open(link)
	contents = f.read()
	f.close()
	opener.close()
		
	soup = BeautifulSoup(''.join(contents))
	links = soup.findAll('source', type = 'video/mp4')
	assert len(links) == 1
	file = links[0]	
	return file.get('src')
	
	'''
	files = []
	for file in soup.findAll('source', type = 'video/mp4'):
		files.append(file.get('src'))
	
	return files
	'''
	
if __name__ == "__main__":
	''' '''
	# for gpm
	# link = "https://class.coursera.org/pgm/lecture/preview"
	# soup = BeautifulSoup(''.join(contents))
	
	# for ml
	# link = "https://class.coursera.org/ml/lecture/preview"
	
	# for nlp
	link = "https://class.coursera.org/nlp/lecture/preview"
	
	soup = fetch_url(link)
	# f = open('contents.html', 'r')
	# contents = f.read()
	# f.close()
		
	# print soup.prettify()	
	links = listLinks(soup)
	# print links	
	f = open('nlp', 'w')
	for link in links:		
		mp4_link = findMP4link(link)		
		f.write(mp4_link)
		f.write("\n")
	f.close()

AttributeError: module 'urllib' has no attribute 'urlopen'

In [9]:
import urllib.request as ur
from bs4 import BeautifulSoup as BS

url = '''https://archive.org/details/20070519_detroit2'''
#open and read page
page = ur.urlopen(url)
html = page.read()
#create BeautifulSoup parse-able "soup"
soup = BS(html)
#get the src attribute from the video tag
video = soup.find("video").get("src")
print(video)

AttributeError: 'NoneType' object has no attribute 'get'

In [48]:
import requests, urllib
import nltk
from bs4 import BeautifulSoup, SoupStrainer
import mimetypes
r = requests.get('https://www.who.int/emergencies/diseases/novel-coronavirus-2019')
soup = BeautifulSoup(r.content,'html.parser')
links = soup.find_all("iframe")
for link in links:
    video_link = link['src']
    req = urllib.request.Request(video_link, method='HEAD', headers={'User-Agent': 'Mozilla/5.0'})
    r = urllib.request.urlopen(req)
    print(video_link)
    print(r.getheader('Content-Type'))

https://www.googletagmanager.com/ns.html?id=GTM-5QFSQRT
text/html; charset=UTF-8
https://player.4am.ch/who/202004-06_pressconf/player.html?lang=en
text/html
https://www.youtube.com/embed/qHksVwOrpbE
text/html; charset=utf-8
https://www.youtube.com/embed/mOV1aBVYKGA
text/html; charset=utf-8


In [36]:
r = requests.get('https://www.who.int/emergencies/diseases/novel-coronavirus-2019')
soup = BeautifulSoup(r.content,'html.parser')
links = soup.findAll('source')
print(links)


[]


In [40]:
r = requests.get('https://www.who.int/emergencies/diseases/novel-coronavirus-2019')
soup = BeautifulSoup(r.content,'html.parser')
links = soup.find_all('iframe', type = 'mp4')
print(links)

[]


In [43]:
import mimetypes
url = 'http://media.theaterchurch.com/podcast/video/hd/720p/2016/05-08-16-720p.mp4'
print(mimetypes.MimeTypes().guess_type(url)[0])

video/mp4


In [3]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
page = requests.get('https://www.gov.uk/coronavirus')    
data = page.text
soup = BeautifulSoup(data)
for link in soup.find_all('a'):
	alink = link.get('href')
	print(alink)

#content
/help/cookies
/help/cookies
/help/cookies
https://www.gov.uk
/coronavirus
/transition
/
/find-coronavirus-local-restrictions
https://www.gov.uk/get-coronavirus-test
/find-coronavirus-support
/government/publications/making-a-christmas-bubble-with-friends-and-family/making-a-christmas-bubble-with-friends-and-family
/guidance/guidance-for-the-christmas-period
/find-coronavirus-local-restrictions
https://www.gov.uk/find-coronavirus-local-restrictions?priority-taxon=774cee22-d896-44c1-a611-e3109cce8eae
/guidance/local-restriction-tiers-what-you-need-to-know?priority-taxon=774cee22-d896-44c1-a611-e3109cce8eae
/guidance/tier-1-medium-alert?priority-taxon=774cee22-d896-44c1-a611-e3109cce8eae
/guidance/tier-2-high-alert?priority-taxon=774cee22-d896-44c1-a611-e3109cce8eae
/guidance/tier-3-very-high-alert?priority-taxon=774cee22-d896-44c1-a611-e3109cce8eae
/guidance/full-list-of-local-restriction-tiers-by-area?priority-taxon=774cee22-d896-44c1-a611-e3109cce8eae
/government/publications/